In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import time
import librairie_qui_remplace_open_cv as k
import ttkbootstrap as tb
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import messagebox

# Interface plus moderne

In [ ]:
import ttkbootstrap as tb
from ttkbootstrap.constants import *
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np

class ImageProcessorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🖼️ Traitement d'Images - Interface Moderne")
        self.root.geometry("1100x700")
        self.root.resizable(False, False)

        self.image = None
        self.image2 = None
        self.processed_image = None
        self.param_entries = {}

        self.algorithm_requirements = {
            "Sobel": {"images": 1, "params": []},
            "Otsu": {"images": 1, "params": []},
            "Nagao": {"images": 1, "params": []},
            "Hough": {"images": 1, "params": []},
            "Transformation lut": {"images": 1, "params": []},
            "Niveaux de gris": {"images": 1, "params": []},
            "Rapport inverse contraste": {"images": 1, "params": []},
            "Étirement d'histogramme": {"images": 1, "params": []},
            "Égalisation d'histogramme": {"images": 1, "params": []},
            "Spécification d'histogramme": {"images": 2, "params": []},
            "Addition": {"images": 2, "params": []},
            "Soustraction": {"images": 2, "params": []},
            "&": {"images": 2, "params": []},
            "|": {"images": 2, "params": []},
            "Compression": {"images": 1, "params": []},
            "Médian": {"images": 1, "params": ["kernel_size"]},
            "Multiplier avec ratio": {"images": 1, "params": ["ratio"]},
            "Convolution": {"images": 1, "params": ["filtre"]},
        }

        self.setup_ui()

    def setup_ui(self):
        self.main_pane = tb.PanedWindow(self.root, orient=HORIZONTAL)
        self.main_pane.pack(fill=BOTH, expand=YES)

        # Sidebar gauche
        self.sidebar = tb.Frame(self.main_pane, padding=10)
        self.main_pane.add(self.sidebar, weight=1)

        self.btn_load = tb.Button(self.sidebar, text="📁 Charger Image", bootstyle="primary-outline", command=self.load_image)
        self.btn_load.pack(fill=X, pady=5)

        self.algorithm_var = tb.StringVar(value="Sobel")
        self.dropdown = tb.Combobox(self.sidebar, textvariable=self.algorithm_var, values=list(self.algorithm_requirements.keys()), state="readonly")
        self.dropdown.pack(fill=X, pady=5)
        self.dropdown.bind("<<ComboboxSelected>>", self.on_algorithm_change)

        self.btn_apply = tb.Button(self.sidebar, text="▶️ Appliquer", bootstyle="success", command=self.apply_algorithm)
        self.btn_apply.pack(fill=X, pady=10)

        self.frame_params = tb.LabelFrame(self.sidebar, text="Paramètres", padding=10)
        self.frame_params.pack(fill=X, pady=5)

        # Zone centrale pour les images
        self.frame_images = tb.Notebook(self.main_pane, bootstyle="primary")
        self.main_pane.add(self.frame_images, weight=4)

        self.tab_original = tb.Frame(self.frame_images)
        self.tab_second = tb.Frame(self.frame_images)
        self.tab_result = tb.Frame(self.frame_images)

        self.label_original = tb.Label(self.tab_original, text="Image originale")
        self.label_original.pack(padx=10, pady=10)

        self.label_second = tb.Label(self.tab_second, text="Deuxième image")
        self.label_second.pack(padx=10, pady=10)

        self.label_result = tb.Label(self.tab_result, text="Résultat")
        self.label_result.pack(padx=10, pady=10)

        self.frame_images.add(self.tab_original, text="🖼️ Original")
        self.frame_images.add(self.tab_second, text="📷 Image 2")
        self.frame_images.add(self.tab_result, text="✅ Résultat")

        self.on_algorithm_change()

    def on_algorithm_change(self, event=None):
        algo = self.algorithm_var.get()
        requirements = self.algorithm_requirements.get(algo, {})

        # Charger 2e image si nécessaire
        if requirements.get("images", 1) == 2:
            self.add_second_image_button()
        else:
            self.remove_second_image_button()

        # Rafraîchir les paramètres
        for widget in self.frame_params.winfo_children():
            widget.destroy()
        self.param_entries = {}

        for param in requirements.get("params", []):
            self.create_param_input(param)

    def add_second_image_button(self):
        if not hasattr(self, 'btn_load_second'):
            self.btn_load_second = tb.Button(self.sidebar, text="📁 Charger 2ème image", bootstyle="secondary-outline", command=self.load_second_image)
            self.btn_load_second.pack(fill=X, pady=5)

    def remove_second_image_button(self):
        if hasattr(self, 'btn_load_second'):
            self.btn_load_second.destroy()
            del self.btn_load_second
            self.image2 = None

    def create_param_input(self, param_key, default_value="3"):
        frame = tb.Frame(self.frame_params)
        frame.pack(fill=X, pady=2)
        label = tb.Label(frame, text=param_key.capitalize(), width=15, anchor="w")
        label.pack(side=LEFT)
        entry = tb.Entry(frame)
        entry.insert(0, str(default_value))
        entry.pack(side=LEFT, fill=X, expand=True)
        self.param_entries[param_key] = entry

    def get_params(self):
        return {k: e.get() for k, e in self.param_entries.items()}

    def load_image(self):
        path = filedialog.askopenfilename()
        if path:
            self.image = k.lireImage(path)
            self.display_image(self.image, self.label_original)
            self.image = k.convertir_en_niveaux_de_gris(self.image)

    def load_second_image(self):
        path = filedialog.askopenfilename()
        if path:
            self.image2 = k.lireImage(path)
            self.display_image(self.image2, self.label_second)
            self.image2 = k.convertir_en_niveaux_de_gris(self.image2)

    def apply_algorithm(self):
        algo = self.algorithm_var.get()
        requirements = self.algorithm_requirements.get(algo, {})

        if self.image is None:
            return messagebox.showerror("Erreur", "Image principale requise.")

        if requirements.get("images", 1) == 2 and self.image2 is None:
            return messagebox.showerror("Erreur", "Deuxième image requise.")

        params = self.get_params()

        try:
            if algo == "Sobel":
                self.processed_image = k.appliquer_filtre_sobel(self.image)
            elif algo == "Otsu":
                self.processed_image = k.methode_otsu(self.image)[0]
            elif algo == "Nagao":
                self.processed_image = k.filtre_nagao(self.image)
            elif algo == "Hough":
                self.processed_image = k.detect_lines_avec_Hough(self.image)
            elif algo == "Transformation lut":
                self.processed_image = k.transformation_lut(self.image)
            elif algo == "Niveaux de gris":
                self.processed_image = k.convertir_en_niveaux_de_gris(self.image)
            elif algo == "Rapport inverse contraste":
                self.processed_image = k.rapport_inverse_contraste(self.image)
            elif algo == "Compression":
                self.processed_image = k.compression_image_bilineaire(self.image, 0.5)
            elif algo == "Étirement d'histogramme":
                self.processed_image = k.etirement_histog(self.image)
            elif algo == "Égalisation d'histogramme":
                self.processed_image = k.egalisation_histogramme(self.image)
            elif algo == "Spécification d'histogramme":
                self.processed_image = k.specification_histogramme(self.image, self.image2)
            elif algo == "Addition":
                self.processed_image = k.addition2images(self.image, self.image2)
            elif algo == "Soustraction":
                self.processed_image = k.soustraction2images(self.image, self.image2)
            elif algo == "&":
                self.processed_image = k.appliquerOperatorET(self.image, self.image2)
            elif algo == "|":
                self.processed_image = k.appliquerOperatorOU(self.image, self.image2)
            elif algo == "Médian":
                kernel_size = int(params.get("kernel_size", 3))
                self.processed_image = k.filtre_median(self.image, kernel_size)
            elif algo == "Convolution":
                self.create_filter_grid(filter_size=3)
                return
            elif algo == "Multiplier avec ratio":
                ratio = int(params.get("ratio", 1))
                self.processed_image = k.multiplier_image_avec_ratio(self.image, ratio)

            self.display_image(self.processed_image, self.label_result)

        except Exception as e:
            messagebox.showerror("Erreur", f"Une erreur est survenue : {e}")

    def display_image(self, img, label):
        if img is None: return
        img = Image.fromarray(np.uint8(img)).resize((350, 350))
        img_tk = ImageTk.PhotoImage(img)
        label.config(image=img_tk)
        label.image = img_tk

    def create_filter_grid(self, filter_size=3):
        for widget in self.frame_params.winfo_children():
            widget.destroy()

        self.filter_entries = []
        for i in range(filter_size):
            row_entries = []
            for j in range(filter_size):
                entry = tb.Entry(self.frame_params, width=5)
                entry.grid(row=i, column=j, padx=2, pady=2)
                row_entries.append(entry)
            self.filter_entries.append(row_entries)

        self.btn_validate_filter = tb.Button(self.frame_params, text="✅ Appliquer filtre", bootstyle="info", command=self.get_filter_matrix_from_entries)
        self.btn_validate_filter.grid(row=filter_size, column=0, columnspan=filter_size, pady=5)

    def get_filter_matrix_from_entries(self):
        try:
            matrix = np.array([[int(e.get()) for e in row] for row in self.filter_entries])
            self.processed_image = k.convolution(self.image, matrix)
            self.display_image(self.processed_image, self.label_result)
        except:
            messagebox.showerror("Erreur", "Filtre invalide.")

if __name__ == "__main__":
    root = tb.Window(themename="cosmo")  # Thème plus moderne
    app = ImageProcessorApp(root)
    root.mainloop()